In [1]:
from google.cloud import storage
import os
import pandas as pd
import numpy as np
import sqlite3
from tqdm.notebook import tqdm

In [2]:
query_data_train = pd.read_json("gs://data_tql/spider/train_spider.json")
query_data_train['dataset'] = 'train'
query_data_train.head()

,db_id,query,query_toks,query_toks_no_value,question,question_toks,sql,dataset
0,department_management,SELECT count(*) FROM head WHERE age > 56,"[SELECT, count, (, *, ), FROM, head, WHERE, ag...","[select, count, (, *, ), from, head, where, ag...",How many heads of the departments are older th...,"[How, many, heads, of, the, departments, are, ...","{'from': {'table_units': [['table_unit', 1]], ...",train
1,department_management,"SELECT name , born_state , age FROM head ORD...","[SELECT, name, ,, born_state, ,, age, FROM, he...","[select, name, ,, born_state, ,, age, from, he...","List the name, born state and age of the heads...","[List, the, name, ,, born, state, and, age, of...","{'from': {'table_units': [['table_unit', 1]], ...",train
2,department_management,"SELECT creation , name , budget_in_billions ...","[SELECT, creation, ,, name, ,, budget_in_billi...","[select, creation, ,, name, ,, budget_in_billi...","List the creation year, name and budget of eac...","[List, the, creation, year, ,, name, and, budg...","{'from': {'table_units': [['table_unit', 0]], ...",train
3,department_management,"SELECT max(budget_in_billions) , min(budget_i...","[SELECT, max, (, budget_in_billions, ), ,, min...","[select, max, (, budget_in_billions, ), ,, min...",What are the maximum and minimum budget of the...,"[What, are, the, maximum, and, minimum, budget...","{'from': {'table_units': [['table_unit', 0]], ...",train
4,department_management,SELECT avg(num_employees) FROM department WHER...,"[SELECT, avg, (, num_employees, ), FROM, depar...","[select, avg, (, num_employees, ), from, depar...",What is the average number of employees of the...,"[What, is, the, average, number, of, employees...","{'from': {'table_units': [['table_unit', 0]], ...",train


In [3]:
query_data_dev = pd.read_json("gs://data_tql/spider/dev.json")
query_data_dev['dataset'] = 'dev'
query_data_dev.head()

,db_id,query,query_toks,query_toks_no_value,question,question_toks,sql,dataset
0,concert_singer,SELECT count(*) FROM singer,"[SELECT, count, (, *, ), FROM, singer]","[select, count, (, *, ), from, singer]",How many singers do we have?,"[How, many, singers, do, we, have, ?]","{'from': {'table_units': [['table_unit', 1]], ...",dev
1,concert_singer,SELECT count(*) FROM singer,"[SELECT, count, (, *, ), FROM, singer]","[select, count, (, *, ), from, singer]",What is the total number of singers?,"[What, is, the, total, number, of, singers, ?]","{'from': {'table_units': [['table_unit', 1]], ...",dev
2,concert_singer,"SELECT name , country , age FROM singer ORDE...","[SELECT, name, ,, country, ,, age, FROM, singe...","[select, name, ,, country, ,, age, from, singe...","Show name, country, age for all singers ordere...","[Show, name, ,, country, ,, age, for, all, sin...","{'from': {'table_units': [['table_unit', 1]], ...",dev
3,concert_singer,"SELECT name , country , age FROM singer ORDE...","[SELECT, name, ,, country, ,, age, FROM, singe...","[select, name, ,, country, ,, age, from, singe...","What are the names, countries, and ages for ev...","[What, are, the, names, ,, countries, ,, and, ...","{'from': {'table_units': [['table_unit', 1]], ...",dev
4,concert_singer,"SELECT avg(age) , min(age) , max(age) FROM s...","[SELECT, avg, (, age, ), ,, min, (, age, ), ,,...","[select, avg, (, age, ), ,, min, (, age, ), ,,...","What is the average, minimum, and maximum age ...","[What, is, the, average, ,, minimum, ,, and, m...","{'from': {'table_units': [['table_unit', 1]], ...",dev


In [4]:
query_data_others = pd.read_json("gs://data_tql/spider/train_others.json")
query_data_others['dataset'] = 'others'
query_data_others.head()

,db_id,query,query_toks,query_toks_no_value,question,question_toks,sql,dataset
0,geo,SELECT city_name FROM city WHERE population =...,"[SELECT, city_name, FROM, city, WHERE, populat...","[select, city_name, from, city, where, populat...",what is the biggest city in wyoming,"[what, is, the, biggest, city, in, wyoming]","{'from': {'table_units': [['table_unit', 1]], ...",others
1,geo,SELECT city_name FROM city WHERE population =...,"[SELECT, city_name, FROM, city, WHERE, populat...","[select, city_name, from, city, where, populat...",what wyoming city has the largest population,"[what, wyoming, city, has, the, largest, popul...","{'from': {'table_units': [['table_unit', 1]], ...",others
2,geo,SELECT city_name FROM city WHERE population =...,"[SELECT, city_name, FROM, city, WHERE, populat...","[select, city_name, from, city, where, populat...",what is the largest city in wyoming,"[what, is, the, largest, city, in, wyoming]","{'from': {'table_units': [['table_unit', 1]], ...",others
3,geo,SELECT city_name FROM city WHERE population =...,"[SELECT, city_name, FROM, city, WHERE, populat...","[select, city_name, from, city, where, populat...",where is the most populated area of wyoming,"[where, is, the, most, populated, area, of, wy...","{'from': {'table_units': [['table_unit', 1]], ...",others
4,geo,SELECT city_name FROM city WHERE population =...,"[SELECT, city_name, FROM, city, WHERE, populat...","[select, city_name, from, city, where, populat...",which city in wyoming has the largest population,"[which, city, in, wyoming, has, the, largest, ...","{'from': {'table_units': [['table_unit', 1]], ...",others


In [5]:
query_data = pd.concat([query_data_train, query_data_dev, query_data_others], axis = 0)
query_data = query_data[['db_id', 'question', 'query', 'dataset']].drop_duplicates()
query_data = query_data.rename(columns = {'question': 'TQL', 'query': 'SQL'})
query_data.shape

(9685, 4)

In [6]:
query_data.drop_duplicates()['db_id'].nunique()

166

In [7]:
def list_blobs(bucket_name, folder_name):
    """List all files in given COS directory."""    
    blob_names = []
    gcs_client = storage.Client()
    bucket = gcs_client.bucket(bucket_name)
    blobs = list(bucket.list_blobs(prefix=folder_name))
    for blob in blobs:
        blob_names.append(blob.name)
    return blob_names
    
def list_blobs_pd(bucket_name, folder_name):
    """List all files in given COS directory."""       
    gcs_client = storage.Client()
    bucket = gcs_client.bucket(bucket_name)
    blobs = list(bucket.list_blobs(prefix=folder_name))

    blob_name = []
    blob_size = []
    blob_time = []
    
    for blob in blobs:
        blob_name.append(blob.name)
        blob_size.append(blob.size)
        blob_time.append(blob.time_created)

    blobs_df = pd.DataFrame(list(zip(blob_name, blob_size, blob_time)), columns=['filePath', 'size', 'timeStamp'])    
    return blobs_df

def download_blob(bucket_name, source_blob_name, destination_file_name):
    """Downloads a blob from COS bucket."""
    gcs_client = storage.Client()
    bucket = gcs_client.bucket(bucket_name)
    blob = bucket.blob(source_blob_name)
    blob.download_to_filename(destination_file_name)

def upload_blob(bucket_name, source_file_name, destination_blob_name):
    """Uploads a file to the bucket."""    
    gcs_client = storage.Client()
    bucket = gcs_client.bucket(bucket_name)
    blob = bucket.blob(destination_blob_name)
    blob.upload_from_filename(source_file_name)

In [8]:
bucket_name = "data_tql"
folder_name = "spider/database"

In [9]:
blobs = list_blobs_pd(bucket_name, folder_name)
blobs['schema'] = blobs['filePath'].apply(lambda x: x.split('/')[2])
blobs['fileName'] = blobs['filePath'].apply(lambda x: x.split('/')[-1])
blobs['extension'] = blobs['fileName'].apply(lambda x: x.split('.')[1])
blobs.head()

,filePath,size,timeStamp,schema,fileName,extension
0,spider/database/academic/academic.sqlite,122880,2023-04-22 02:03:27.321000+00:00,academic,academic.sqlite,sqlite
1,spider/database/academic/schema.sql,2264,2023-04-22 02:03:26.785000+00:00,academic,schema.sql,sql
2,spider/database/activity_1/activity_1.sqlite,24576,2023-04-22 02:06:11.982000+00:00,activity_1,activity_1.sqlite,sqlite
3,spider/database/activity_1/schema.sql,16346,2023-04-22 02:06:11.712000+00:00,activity_1,schema.sql,sql
4,spider/database/aircraft/aircraft.sqlite,45056,2023-04-22 02:04:17.631000+00:00,aircraft,aircraft.sqlite,sqlite


In [10]:
sqlite_dict = {}
no_db_file = []

for schema in blobs.schema.unique():
    df_schema = blobs[blobs['schema'] == schema]
    if 'sqlite' in df_schema['extension'].tolist():
        filePath = df_schema[df_schema['extension'] == 'sqlite']['filePath'].values[0]
        fileName = df_schema[df_schema['extension'] == 'sqlite']['fileName'].values[0]
        sqlite_dict[schema] = [filePath, fileName]
    else:
        no_db_file.append(schema)

In [11]:
blobs_sqlite = blobs[blobs['extension'] == 'sqlite'][['filePath', 'schema', 'fileName']]
query_data_blobs = query_data.merge(blobs_sqlite, left_on = 'db_id', right_on = 'schema')
query_data_blobs = query_data_blobs.drop(['filePath', 'schema'], axis = 1)
query_data_blobs['filePath'] = 'sqliteDB/' + query_data_blobs['fileName']
query_data_blobs['result'] = np.nan
query_data_blobs

,db_id,TQL,SQL,dataset,fileName,filePath,result
0,department_management,How many heads of the departments are older th...,SELECT count(*) FROM head WHERE age > 56,train,department_management.sqlite,sqliteDB/department_management.sqlite,NaN
1,department_management,"List the name, born state and age of the heads...","SELECT name , born_state , age FROM head ORD...",train,department_management.sqlite,sqliteDB/department_management.sqlite,NaN
2,department_management,"List the creation year, name and budget of eac...","SELECT creation , name , budget_in_billions ...",train,department_management.sqlite,sqliteDB/department_management.sqlite,NaN
3,department_management,What are the maximum and minimum budget of the...,"SELECT max(budget_in_billions) , min(budget_i...",train,department_management.sqlite,sqliteDB/department_management.sqlite,NaN
4,department_management,What is the average number of employees of the...,SELECT avg(num_employees) FROM department WHER...,train,department_management.sqlite,sqliteDB/department_management.sqlite,NaN
...,...,...,...,...,...,...,...
9680,restaurants,where can i find a restaurant in the bay area ?,"SELECT t2.house_number , t1.name FROM LOCATI...",others,restaurants.sqlite,sqliteDB/restaurants.sqlite,NaN
9681,restaurants,give me a good restaurant in the bay area ?,"SELECT t2.house_number , t1.name FROM LOCATI...",others,restaurants.sqlite,sqliteDB/restaurants.sqlite,NaN
9682,restaurants,what are some good restaurants in the bay area ?,"SELECT t2.house_number , t1.name FROM LOCATI...",others,restaurants.sqlite,sqliteDB/restaurants.sqlite,NaN
9683,restaurants,give me some good restaurants in the bay area ?,"SELECT t2.house_number , t1.name FROM LOCATI...",others,restaurants.sqlite,sqliteDB/restaurants.sqlite,NaN


In [12]:
all_blobs = list_blobs(bucket_name, 'sqliteDB')
failed_queries = {}
if not no_db_file:
    
    for schema in tqdm(sqlite_dict.keys()):
        filePath = sqlite_dict[schema][0]
        fileName = sqlite_dict[schema][1]
        download_blob(bucket_name, filePath, fileName)
        
        if f'sqliteDB/{fileName}' not in all_blobs:
            upload_blob(bucket_name, fileName, f'sqliteDB/{fileName}')
            
        schema_queries = query_data_blobs[query_data_blobs['db_id'] == schema]        
        if len(schema_queries) != 0:
            
            conn = sqlite3.connect(f'{schema}.sqlite')
            cursor = conn.cursor()
            schema_queries = schema_queries[['SQL']].drop_duplicates().values
            
            for schema_query in schema_queries:
                number_of_same_queries = len(query_data_blobs[(query_data_blobs['db_id'] == schema) & (query_data_blobs['SQL'] == schema_query[0])])
                try:
                    res_dict = pd.read_sql_query(schema_query[0], conn).to_dict()
                    query_data_blobs.loc[(query_data_blobs['db_id'] == schema) &
                                         (query_data_blobs['SQL'] == schema_query[0]), 'result'] = [res_dict] * number_of_same_queries
                    
                except Exception as err:
                    failed_queries[schema] = [schema_query[0], err]
            conn.close()
        os.remove(fileName)

  0%|          | 0/166 [00:00<?, ?it/s]

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:21: UserWarning: DataFrame columns are not unique, some columns will be omitted.


In [13]:
failed_queries_df = pd.DataFrame(failed_queries).T.reset_index()
failed_queries_df.columns = ['db_id', 'SQL', 'Error']
display(failed_queries_df)
failed_queries_df.to_csv('gs://data_tql/spider/processed/spiderFailedQueries.csv', index = False)

,db_id,SQL,Error
0,assets_maintenance,SELECT T1.company_name FROM Third_Party_Compan...,Execution failed on sql 'SELECT T1.company_nam...
1,document_management,SELECT document_name FROM documents GROUP BY d...,Execution failed on sql 'SELECT document_name ...
2,wta_1,"SELECT first_name , last_name FROM players OR...",Could not decode to UTF-8 column 'last_name' w...


In [14]:
display(query_data_blobs)
query_data_blobs.to_csv('gs://data_tql/spider/processed/spiderQueryData.csv', index = False)

,db_id,TQL,SQL,dataset,fileName,filePath,result
0,department_management,How many heads of the departments are older th...,SELECT count(*) FROM head WHERE age > 56,train,department_management.sqlite,sqliteDB/department_management.sqlite,{'count(*)': {0: 5}}
1,department_management,"List the name, born state and age of the heads...","SELECT name , born_state , age FROM head ORD...",train,department_management.sqlite,sqliteDB/department_management.sqlite,"{'name': {0: 'Pádraig Harrington', 1: 'Stewart..."
2,department_management,"List the creation year, name and budget of eac...","SELECT creation , name , budget_in_billions ...",train,department_management.sqlite,sqliteDB/department_management.sqlite,"{'Creation': {0: '1789', 1: '1789', 2: '1947',..."
3,department_management,What are the maximum and minimum budget of the...,"SELECT max(budget_in_billions) , min(budget_i...",train,department_management.sqlite,sqliteDB/department_management.sqlite,"{'max(budget_in_billions)': {0: 543.2}, 'min(b..."
4,department_management,What is the average number of employees of the...,SELECT avg(num_employees) FROM department WHER...,train,department_management.sqlite,sqliteDB/department_management.sqlite,{'avg(num_employees)': {0: 105468.16666666667}}
...,...,...,...,...,...,...,...
9680,restaurants,where can i find a restaurant in the bay area ?,"SELECT t2.house_number , t1.name FROM LOCATI...",others,restaurants.sqlite,sqliteDB/restaurants.sqlite,"{'HOUSE_NUMBER': {}, 'NAME': {}}"
9681,restaurants,give me a good restaurant in the bay area ?,"SELECT t2.house_number , t1.name FROM LOCATI...",others,restaurants.sqlite,sqliteDB/restaurants.sqlite,"{'HOUSE_NUMBER': {}, 'NAME': {}}"
9682,restaurants,what are some good restaurants in the bay area ?,"SELECT t2.house_number , t1.name FROM LOCATI...",others,restaurants.sqlite,sqliteDB/restaurants.sqlite,"{'HOUSE_NUMBER': {}, 'NAME': {}}"
9683,restaurants,give me some good restaurants in the bay area ?,"SELECT t2.house_number , t1.name FROM LOCATI...",others,restaurants.sqlite,sqliteDB/restaurants.sqlite,"{'HOUSE_NUMBER': {}, 'NAME': {}}"
